In [1]:
# from txtai import LLM
import requests
import sqlite3
import csv
from tqdm import tqdm
import time
import requests
import os
import pandas
import pandas as pd
import json

In [2]:
def GPT(prompt: str, model: str, **kwargs):
    import requests
    import json
    url = "https://gpt-api.hkust-gz.edu.cn/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "9c77caa0882f46a19381a4796a641181556d0fe4e6c145bdb3717f5936207cae"
    }
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.1
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    res = json.loads(response.text)

    if res["choices"][0]["message"]["content"] is not None:
        return res["choices"][0]["message"]["content"]
    else:
        return 'None'

In [3]:
def get_wikidata_id(search_term):
    """Fetch the Wikidata identifier for a given search term."""
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "language": "en",
        "format": "json",
        "search": search_term
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if data['search']:
            return data['search'][0]['id']
        else:
            return None
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def name_belongs_to_entity(wikidata_id, topicid):
    """Checks if the entity with the given Wikidata ID is an ACM Fellow."""
    query = """
    SELECT ?award WHERE {
      wd:%s p:P166 ?statement.
      ?statement ps:P166 ?award.
      FILTER(?award = wd:%s)
    }
    """ % (wikidata_id,topicid)

    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    params = {"query": query, "format": "json"}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        return len(data['results']['bindings']) > 0
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return False


def search_wikidata_property(keyword):
    """
    Search for a Wikidata property ID based on a keyword.
    Returns a list of matching properties with their IDs and descriptions.
    """
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "language": "en",
        "type": "property",
        "format": "json",
        "search": keyword
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX/5XX
        data = response.json()

        properties = []
        prop_id = ''
        for result in data.get("search", []):
            prop_id = result.get("id")
            label = result.get("label")
            description = result.get("description", "No description available.")
            properties.append((prop_id, label, description))

        return properties
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [4]:
def get_entities_wikipedia_titles(property_id, value_id):
    """Fetches Wikipedia titles for entities based on a specified property and its value."""
    query = f"""
      SELECT ?entity ?entityLabel ?article WHERE {{
      ?entity p:{property_id} ?statement.
      ?statement ps:{property_id} wd:{value_id}.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
      OPTIONAL {{
        ?article schema:about ?entity;
                 schema:inLanguage "en";
                 schema:isPartOf <https://en.wikipedia.org/>.
      }}
    }}
    """
    url = "https://query.wikidata.org/sparql"
    headers = {"Accept": "application/sparql-results+json"}
    params = {"query": query}

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        titles = []
        for item in data['results']['bindings']:
            if 'article' in item:
                article_url = item['article']['value']
                title = article_url.split('/')[-1]  # Extract Wikipedia title from URL
                titles.append(title)

        return titles
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return []

def get_wikipedia_first_paragraph(title):
    """Fetches the first paragraph of a Wikipedia page for a given title."""
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        page = next(iter(data['query']['pages'].values()))
        if 'extract' in page:
            content = page['extract']
            first_paragraph = content.split('\n')[0]  # Assuming the first paragraph is before the first newline
            return first_paragraph
        else:
            return "Content not available."
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return "Error retrieving content."

# Example usage
topic = "Nobel Prize in Chemistry"  # ACM fellow, city, President of the United States, Chemical Elements, Summer Olympic Games,  Nobel Prize in Chemistry
keyword = "award received" # award received, instance of, position held, instance of, instance of,award received

topicid = get_wikidata_id(topic)
property_id = search_wikidata_property(keyword)[0][0]
print(topicid)
print(property_id)
# property_id = "P54"  # Example: Wikidata property for "award received" P166
# value_id = "Q18748039"  # Example: Wikidata item for "ACM Fellow"
titles = get_entities_wikipedia_titles(property_id, topicid)
print(len(titles))
# for title in titles:  # Limiting to first 5 results for demonstration
#     first_paragraph = get_wikipedia_first_paragraph(title)
#     print(f"Title: {title}\nFirst Paragraph: {first_paragraph}\n")

Q44585
P166
197


In [5]:
def csv_sqltool(table, sql_query):
    try:
        csv_file_path = "{}.csv".format(table)
        df = pd.read_csv(csv_file_path)

        # Create a connection to a new in-memory SQLite database
        conn = sqlite3.connect(':memory:')

        # Copy the DataFrame data into a new SQLite table
        df.to_sql(table, conn, if_exists='replace', index=False)

        # Execute the SQL query
        result_df = pd.read_sql_query(sql_query, conn)

        # Print the result DataFrame
        # print('reults = ', result_df)
        columns = ', '.join(result_df.columns)
        rows = []
        for row in result_df.itertuples(index=False, name=None):
            # print('row = ',row)
            rows.append(row)
        # print(rows)
        return columns, rows
    except pandas.errors.DatabaseError as e:
        print("数据库操作出错:", e)
        print("An error occurred:", e)
        return 'Wrong'
    

In [6]:
def run(query, topic):

    # fout_entity = open(os.path.join('entities_first_paragraph.jsonl'), 'a')
    print("------Search start!")
    prompt = '''The query is {}, if we search the query in wikidata, what properties related to {} should we choose? just output the property words in a format 'property:...\n property:...\n ...'.no other word and symbol.'''.format(query, topic)
    answer = GPT(prompt, "GPT-4")
    # print(answer)
    alltitles = []
    keywords = []
    entity_i = []
    topicid = get_wikidata_id(topic)
    for line in answer.split('\n'):
         if "property:" in line:
            keyword = line.split(':')[1]
            print(keyword)
            if len(search_wikidata_property(keyword))>0:
                property_id = search_wikidata_property(keyword)[0][0]
                titles = get_entities_wikipedia_titles(property_id, topicid)
            if len(titles) != 0:
                keywords.append(keyword)
                alltitles = list(set(alltitles) | set(titles))
    if len(alltitles) == 0:
        titles = get_entities_wikipedia_titles('P463', topicid)
        if len(titles) != 0:
            keywords.append('member of')
            alltitles = list(set(alltitles) | set(titles))
        titles = get_entities_wikipedia_titles('P166', topicid)
        if len(titles) != 0:
            keywords.append('award received')
            alltitles = list(set(alltitles) | set(titles))
        titles = get_entities_wikipedia_titles('P31', topicid)
        if len(titles) != 0:
            keywords.append('instance of')
            alltitles = list(set(alltitles) | set(titles))
        titles = get_entities_wikipedia_titles('P39', topicid)
        if len(titles) != 0:
            keywords.append('position held')
            alltitles = list(set(alltitles) | set(titles))
        
    #member of: P463
    table_n = topic.replace(' ','').replace('-','')

    progress_bar = tqdm(total=len(alltitles))
    
    for title in alltitles:  # Limiting to first 5 results for demonstration
        first_paragraph = get_wikipedia_first_paragraph(title)
        print(title)
        # print(f"Title: {title}\nFirst Paragraph: {first_paragraph}\n")
        entity_i.append({'title':title, 'firstparagraph':first_paragraph})
        progress_bar.update(1)

    progress_bar.close()
    print("------Search is done!")
    print('Total entities number:', len(alltitles))
    # topic_i =  {'table': table_n, 'edge': keywords,
    #                               'properties': prope, 'entity':entity_i}
    # fout_entity.write(json.dumps(topic_i) + '\n')
   
   
    schema_prompt = '''The query is {}, if we have a table named {}, what should be the columns in the table so we can write a SQL to solve the query? just output the columns and SQL in a format 'columns:...\n SQL:...\n'.no other word and symbol.'''.format(query, table_n)
    schema_answer = GPT(schema_prompt, "GPT-4")
    # print(schema_answer)
    columns = ['Entity']
    sqll = ''
    for line in schema_answer.split('\n'):
        # print(line)
        if "columns:" in line:
            columns_t = line.split(':')[1] 
            for col in columns_t.split(','):
                columns.append(col)
        if "SQL:" in line:
            sqll = line.split(':')[1]
            
    print("------Data transformation start!")
   
    conn = sqlite3.connect('document_store.db')
    cursor = conn.cursor()
    # cursor.close()
    # conn.close()
    cursor.execute(f'DROP TABLE IF EXISTS {table_n}')
    columns_with_types = ', '.join([f'{col} TEXT' for col in columns])
    # print('columns_with_types= ', columns_with_types)
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {table_n} (
      {columns_with_types}
    )
    ''')

    progress_bar_1 = tqdm(total=len(entity_i))
    for entity in entity_i: 

        title = entity['title']
        first_paragraph = entity['firstparagraph']
        data_prompt = f"""
                    Extracting property value using only the context below.If the property value is not mentioned in the context, using 'None'to fill.
                    The output size must meet the size of property. Just output the property value in order one by one in a format '...|...'. 
                    property: {columns[1:]}
                    context: {first_paragraph}
                    """
        data = GPT(data_prompt, "GPT-3.5-turbo")
        # print(data)
        value_i = data.replace('[','').replace(']','').replace('\xf6','').split('|')
        values = ['\'' + title + '\'']
        for value in value_i:
            values.append('\'' + value + '\'')
            
        # print(columns)
        # print(values)
        
        if len(values) == len(columns):
            # print(len(value_i))
            # print(len(property_i))
            sql = f'INSERT INTO {table_n} ({", ".join(columns)}) VALUES ({", ".join(values)})'
            # print(sql)
            cursor.execute(sql)
            # context.join(data)
        progress_bar_1.update(1)

    progress_bar_1.close()
    
    cursor.execute(f'SELECT * FROM {table_n}')
    # Fetch all rows from the query result
    rows = cursor.fetchall()
    # rows = rows
    # Close the cursor and connection to the database
    cursor.close()
    conn.close()
    
    # Write the fetched data to a CSV file
    with open(f'{table_n}.csv', 'w', newline='') as file:
        csv_writer = csv.writer(file)
        # Write the header row if needed
        csv_writer.writerow([column[0] for column in cursor.description])
        # Write the data rows
        csv_writer.writerows(rows)
        
    
    print("------CSV file has been created successfully. Data transformation Done!")
    # cursor.close()
    # conn.close()

    r_answe = csv_sqltool(table_n, sqll)
    # print(r_answe)
    
    ans_prompt = '''
    The question is {}, the provided table information is {}, just give out the answer for the question.
    '''.format(query, r_answe)
    fansw = GPT(ans_prompt, "GPT-4")
    print('The answer is: ', fansw)


In [7]:
def main():

    print_user = True # Controls whether the prompt input field is entered

    topic = 'ACM fellow' 
    query= 'How many ACM fellows are from MIT?'
    
    run(query, topic)
        

In [ ]:
if __name__ == '__main__':
    main()

------Search start!
educated at
award received


  0%|                                                                                 | 1/1123 [00:01<22:40,  1.21s/it]

Wendy_Kellogg


  0%|▏                                                                                | 2/1123 [00:02<21:33,  1.15s/it]

Nicholas_Higham


...

An error occurred: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=None)
Ihab_Ilyas


 51%|██████████████████████████████████████                                    | 577/1123 [1:19:03<11:56:57, 78.79s/it]

An error occurred: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=None)
Alan_Jay_Smith


 51%|██████████████████████████████████████                                    | 578/1123 [1:20:20<11:50:44, 78.25s/it]

An error occurred: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Read timed out. (read timeout=None)
Orna_Grumberg
